In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn 
import pandas as pd
import os
import sys
import time
from torch.autograd import Variable
import re
print(sys.version_info)
for module in mpl,np,sklearn,torch,re:
    print(module.__name__,module.__version__)

sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.2
sklearn 0.22.2.post1
torch 1.4.0+cpu
re 2.2.1


In [2]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1 , self).__init__()
        self.conv=nn.Conv2d(1,10,kernel_size=3, padding=0,bias=False)
        #self.bn=nn.BatchNorm2d(10)
        self.layer1=torch.nn.Linear(6, 10)
        
        self.layerF=nn.ReLU()
        self.layer2=torch.nn.Linear(10, 1,bias=False)
        
        self._initialize_weights()
    def forward(self,x):
        x=self.conv(x)
        x=self.bn(x)
        x=self.layer1(x)
        x=self.layerF(x)
        x=self.layer2(x)
        return x
    def _initialize_weights(self):
        for m in self.modules():  # 提取网络的层次结构
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                print('kernel_size[0]',m.kernel_size[0],'kernel_size[1]:', m.kernel_size[1],'out_channels:',m.out_channels)
                m.weight=nn.Parameter(torch.Tensor(np.ones([m.out_channels,m.in_channels,m.kernel_size[0],m.kernel_size[1]])))
            elif isinstance(m, nn.Linear):
                m.weight=nn.Parameter(torch.Tensor(np.ones([m.out_features,m.in_features])))
     

In [3]:
model=Net1()

kernel_size[0] 3 kernel_size[1]: 3 out_channels: 10


In [4]:
print(model)

Net1(
  (conv): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (layer1): Linear(in_features=6, out_features=10, bias=True)
  (layerF): ReLU()
  (layer2): Linear(in_features=10, out_features=1, bias=False)
)


In [5]:
for m in model.modules():
    print('=============')
    
    print(m)
    print(type(m))
    if isinstance(m , nn.Conv2d) :
        print(m.weight.data.size())
    elif isinstance(m , nn.BatchNorm2d) :
        print(m.weight.data.size())
        print(m.bias.data.size())

Net1(
  (conv): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (layer1): Linear(in_features=6, out_features=10, bias=True)
  (layerF): ReLU()
  (layer2): Linear(in_features=10, out_features=1, bias=False)
)
<class '__main__.Net1'>
Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), bias=False)
<class 'torch.nn.modules.conv.Conv2d'>
torch.Size([10, 1, 3, 3])
Linear(in_features=6, out_features=10, bias=True)
<class 'torch.nn.modules.linear.Linear'>
ReLU()
<class 'torch.nn.modules.activation.ReLU'>
Linear(in_features=10, out_features=1, bias=False)
<class 'torch.nn.modules.linear.Linear'>


### model_dict() 与 model.modules() 与 model.parameters()


In [6]:
model_dict=model.state_dict()

In [31]:
model_dict['layer1.weight']=torch.ones(10,6)

In [29]:
import copy
model_dict1=copy.deepcopy(model.state_dict())

In [24]:
for key in model_dict1:
    print(key)
    print(model_dict1[key])

conv.weight
tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]])
layer1.weight
tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1.

In [34]:
model_dict['layer1.weight']

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

In [32]:
model.load_state_dict(model_dict)

<All keys matched successfully>

In [7]:
#print(model_dict)

print(type(model_dict))
print(type(model_dict.items()))
print(model_dict.keys())

<class 'collections.OrderedDict'>
<class 'odict_items'>
odict_keys(['conv.weight', 'layer1.weight', 'layer1.bias', 'layer2.weight'])


In [8]:
keys=model_dict.keys()

In [9]:
for key in keys:
    print(key)
    if re.search("mean",key) or re.search("_var",key):
        print(model_dict[key])

conv.weight
layer1.weight
layer1.bias
layer2.weight


In [10]:
for m in model.modules():
    print('=============')
    
    print(m)
    if isinstance(m , nn.Conv2d) :
        print(type(m.weight.data))


Net1(
  (conv): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (layer1): Linear(in_features=6, out_features=10, bias=True)
  (layerF): ReLU()
  (layer2): Linear(in_features=10, out_features=1, bias=False)
)
Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), bias=False)
<class 'torch.Tensor'>
Linear(in_features=6, out_features=10, bias=True)
ReLU()
Linear(in_features=10, out_features=1, bias=False)


In [19]:
for param in model.parameters():
     print(param, param.size())

Parameter containing:
tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]], requires_grad=True) torch.Size([10, 1, 3, 3])
Parameter containing:
tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1.